# 0.Set up

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd "/content/drive/MyDrive/cibi_prj/"

/content/drive/MyDrive/cibi_prj


# 0. AntiBerta Demo

In [4]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
from transformers import (
    RobertaTokenizer,
    RobertaForTokenClassification,
    Trainer,
    TrainingArguments,
    AutoTokenizer
)
from datasets import (
    Dataset,
    DatasetDict,
    Sequence,
    ClassLabel
)
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef,
    roc_auc_score,
    average_precision_score
)
import pandas as pd
import torch
import numpy as np
import random
import os

In [ ]:
TOKENIZER_DIR = "./antiberta/antibody-tokenizer"

# Initialise a tokenizer
tokenizer = RobertaTokenizer.from_pretrained(TOKENIZER_DIR, max_len=150)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


In [ ]:
# Read in parquet files
train_df = pd.read_parquet(
    './antiberta/assets/sabdab_train.parquet'
)
val_df = pd.read_parquet(
    './antiberta/assets/sabdab_val.parquet'
)
test_df = pd.read_parquet(
    './antiberta/assets/sabdab_test.parquet'
)

In [ ]:
train_df

,sequence,paratope_labels,paratope_sequence,v_gene,j_gene,pdb,antibody_chains,antigen_type_max,compound,paratope_count_bin,v_gene_cluster
1002,DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNNKNYLAWYQQKP...,"[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",------------------------------Y------Y--------...,IGKV4-1,IGKJ2,7k9z,BA,protein,Crystal structure of SARS-CoV-2 receptor bindi...,1,VK9
511,SALTQPPSVSGAPGQRVTISCTGSSSNIGAGYDVHWYQQLPGTAPK...,"[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",-----------------------------AGYD-------------...,IGLV1-40,IGLJ7,5ush,DE,protein,Structure of vaccinia virus D8 protein bound t...,0,VL8
911,VLTQPPSASGTPGQRVTISCSGSNSNIATNYVCWYQQYPGTAPKPL...,"[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",----------------------------TNY---------------...,IGLV1-47,IGLJ3,6xqw,HL,protein,Crystal Structure of MaliM03 Fab in complex wi...,1,VL8
909,EIVLTQSPGTLSLSPGERATLSCRASQIIDNKYLAWYQQKPGQAPR...,"[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",----------------------------------------------...,IGKV3-20,IGKJ5,6xq0,BC,protein,Human antibody S8V2-18 in complex with the inf...,0,VK8
569,QPVLTQSPSASGTPGQRVTISCSGSSSNIGNNYVYWYQQLPGTAPK...,"[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",-------------------------------NY-Y-----------...,IGLV1-47,IGLJ7,5yy5,HL,protein,Structural definition of a unique neutralizati...,1,VL8
...,...,...,...,...,...,...,...,...,...,...,...
1041,QVQLVQSGAEVKKPGSSVRVSCKASGGTFSTYPISWVRQAPGQGLE...,"[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",-------------------------GGT-STY--------------...,IGHV1-69,IGHJ4,7mf1,HL,protein,Crystal structure of SARS-CoV-2 receptor bindi...,1,VH8
937,QVQLVESGGGLIQPGGSLRLSCAASGLTVNRNYMSWIRQAPGKGLE...,"[N, P, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",-V-----------------------GLT-NRNY-------------...,IGHV3-53,IGHJ3,7bem,HL,protein,Crystal structure of the receptor binding doma...,1,VH5
875,EVQLVESGGGLVKPGGSLRLSCEASGFLFSSYSMNWVRQAPGKGLE...,"[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",---------------------------L--S-S-------------...,IGHV3-21,IGHJ2,6woz,EF,protein,Plasmodium vivax reticulocyte binding protein ...,1,VH5
6,GVQLVESGGGVVQPGRSLRLSCAASGFTFSTYAMHWVRQAPGKGLE...,"[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",------------------------------T-A-------------...,IGHV3-30,IGHJ2,1nl0,HL,peptide,Crystal structure of human factor IX Gla domai...,1,VH5


In [ ]:
ab_dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df[['sequence','paratope_labels']]),
    "validation": Dataset.from_pandas(val_df[['sequence','paratope_labels']]),
    "test": Dataset.from_pandas(test_df[['sequence','paratope_labels']])
})S
ab_dataset

In [ ]:
ab_dataset['train'].select(range(1))['sequence']

['DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNNKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSTPPTFGQGTKLEIK']

In [ ]:
print(ab_dataset['train'].select(range(1))['paratope_labels'][0][:])

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'P', 'N', 'N', 'N', 'N', 'N', 'N', 'P', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'P', 'P', 'P', 'P', 'N', 'P', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']


In [ ]:
ab_dataset['train'].features

{'sequence': Value(dtype='string', id=None),
 'paratope_labels': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [ ]:
paratope_class_label = ClassLabel(2, names=['N','P'])
new_feature = Sequence(
    paratope_class_label
)

In [ ]:
ab_dataset_featurised = ab_dataset.map(
    lambda seq, labels: {
        "sequence": seq,
        "paratope_labels": [paratope_class_label.str2int(sample) for sample in labels]
    },
    input_columns=["sequence", "paratope_labels"], batched=True
)


Map:   0%|          | 0/720 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

In [ ]:
feature_set_copy = ab_dataset['train'].features.copy()
feature_set_copy


{'sequence': Value(dtype='string', id=None),
 'paratope_labels': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [ ]:
feature_set_copy['paratope_labels'] = new_feature

In [ ]:
ab_dataset_featurised = ab_dataset_featurised.cast(feature_set_copy)

Casting the dataset:   0%|          | 0/720 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/90 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/90 [00:00<?, ? examples/s]

In [ ]:
ab_dataset_featurised['train'].features
print(ab_dataset_featurised['train'].select(range(1))['paratope_labels'][0][20:35])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [ ]:
def preprocess(batch):
    # Step 1: Tokenize the 'sequence' using the tokenizer defined earlier.
    t_inputs = tokenizer(batch['sequence'], padding="max_length")
    batch['input_ids'] = t_inputs.input_ids
    batch['attention_mask'] = t_inputs.attention_mask

    # Step 2: Process the paratope_labels so they match the length of input_ids (after tokenization)
    labels_container = []

    # Iterate over each sequence and its corresponding paratope_labels in the batch
    for index, labels in enumerate(batch['paratope_labels']):
        tokenized_input_length = len(batch['input_ids'][index])
        paratope_label_length  = len(batch['paratope_labels'][index])
        # Step 3: Calculate how many padding tokens (including EOS) need to be added to match input_ids length
        n_pads_with_eos = max(1, tokenized_input_length - paratope_label_length - 1)
        # Step 4: Create padded labels with -100 values
        labels_padded = [-100] + labels + [-100] * n_pads_with_eos
        # Step 5: Assert to check that the padded label length matches the input_ids length
        assert len(labels_padded) == len(batch['input_ids'][index]), \
        f"Lengths don't align, {len(labels_padded)}, {len(batch['input_ids'][index])}, {len(labels)}"
        labels_container.append(labels_padded)
    # Step 6: Add a new 'labels' column to the batch, which will be used by the Hugging Face Trainer
    batch['labels'] = labels_container
    # Step 7: Final check to ensure that input_ids, labels, and attention_mask have the same lengths
    for i,v in enumerate(batch['labels']):
        assert len(batch['input_ids'][i]) == len(v) == len(batch['attention_mask'][i])
    return batch

In [ ]:
ab_dataset_tokenized = ab_dataset_featurised.map(
    preprocess,
    batched=True,
    batch_size=8,
    remove_columns=['sequence', 'paratope_labels']
)


Map:   0%|          | 0/720 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

In [ ]:
label_list = paratope_class_label.names

def compute_metrics(p):
    """
    A callback added to the trainer so that we calculate various metrics via sklearn
    """
    predictions, labels = p

    # The predictions are logits, so we apply softmax to get the probabilities. We only need
    # the probabilities of the paratope label, which is index 1 (according to the ClassLabel we made earlier),
    # or the last column from the output tensor
    prediction_pr = torch.softmax(torch.from_numpy(predictions), dim=2).detach().numpy()[:,:,-1]

    # We run an argmax to get the label
    predictions = np.argmax(predictions, axis=2)

    # Only compute on positions that are not labelled -100
    preds = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    labs = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    probs = [
        [prediction_pr[i][pos] for (pr, (pos, l)) in zip(prediction, enumerate(label)) if l!=-100]
         for i, (prediction, label) in enumerate(zip(predictions, labels))
    ]

    # flatten
    preds = sum(preds, [])
    labs = sum(labs, [])
    probs = sum(probs,[])

    return {
        "precision": precision_score(labs, preds, pos_label="P"),
        "recall": recall_score(labs, preds, pos_label="P"),
        "f1": f1_score(labs, preds, pos_label="P"),
        "auc": roc_auc_score(labs, probs),
        "aupr": average_precision_score(labs, probs, pos_label="P"),
        "mcc": matthews_corrcoef(labs, preds),
    }


In [ ]:
batch_size = 32
RUN_ID = "paratope-prediction-task"
SEED = 0
LR = 1e-6

args = TrainingArguments(
    f"{RUN_ID}_{SEED}", # this is the name of the checkpoint folder
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=LR, # 1e-6, 5e-6, 1e-5. .... 1e-3
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    warmup_ratio=0, # 0, 0.05, 0.1 ....
    load_best_model_at_end=True,
    lr_scheduler_type='linear',
    metric_for_best_model='aupr', # name of the metric here should correspond to metrics defined in compute_metrics
    logging_strategy='epoch',
    seed=SEED
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def set_seed(seed: int = 42):
    """
    Set all seeds to make results reproducible (deterministic mode).
    When seed is None, disables deterministic mode.
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)



In [ ]:
import torch
from transformers import RobertaForTokenClassification, AutoTokenizer, Trainer

MODEL_DIR = "./test/checkpoint-5000" # Use pre-trained MLM model

model = RobertaForTokenClassification.from_pretrained(MODEL_DIR, num_labels=2)

# Khởi tạo trainer
trainer = Trainer(
    model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=ab_dataset_tokenized['train'],
    eval_dataset=ab_dataset_tokenized['validation'],
    compute_metrics=compute_metrics
)


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at ./test2/checkpoint-5000 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Auc,Aupr,Mcc
1,0.493700,0.335340,0.000000,0.000000,0.000000,0.467882,0.088214,0.000000
2,0.328500,0.297010,0.000000,0.000000,0.000000,0.670495,0.160353,0.000000
3,0.299700,0.275625,0.000000,0.000000,0.000000,0.775830,0.233652,0.000000
4,0.281900,0.261749,0.000000,0.000000,0.000000,0.814945,0.285729,0.000000
5,0.271900,0.252282,0.000000,0.000000,0.000000,0.833607,0.324603,0.000000
6,0.262700,0.245726,0.444444,0.004103,0.008130,0.844240,0.350840,0.035376
7,0.256200,0.241082,0.680000,0.017436,0.034000,0.851003,0.369007,0.098685
8,0.253000,0.238163,0.631579,0.024615,0.047384,0.855185,0.380230,0.111682
9,0.251100,0.236491,0.617021,0.029744,0.056751,0.857458,0.385784,0.120893
10,0.249500,0.235940,0.620000,0.031795,0.060488,0.858211,0.387962,0.125420


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defin

TrainOutput(global_step=230, training_loss=0.29481027851934016, metrics={'train_runtime': 286.6338, 'train_samples_per_second': 25.119, 'train_steps_per_second': 0.802, 'total_flos': 551172846240000.0, 'train_loss': 0.29481027851934016, 'epoch': 10.0})

In [ ]:
pred = trainer.predict(
    ab_dataset_tokenized['test']
)

In [ ]:
pred.metrics

{'test_loss': 0.23579977452754974,
 'test_precision': 0.6909090909090909,
 'test_recall': 0.03877551020408163,
 'test_f1': 0.07342995169082125,
 'test_auc': 0.8619914068592224,
 'test_aupr': 0.4028475447123681,
 'test_mcc': 0.1488046484884488,
 'test_runtime': 1.05,
 'test_samples_per_second': 85.716,
 'test_steps_per_second': 2.857}

In [ ]:
trainer.save_model()

## Inference - how to go from sequence to predicted sequence


In [ ]:
# input sequence of tralokinumab Light chain
input_seq = 'YVLTQPPSVSVAPGKTARITCGGNIIGSKLVHWYQQKPGQAPVLVIYDDGDRPSGIPERFSGSNSGNTATLTISRVEAGDEADYYCQVWDTGSDPVVFGGGTKLTVL'
model = RobertaForTokenClassification.from_pretrained(
    f"{RUN_ID}_{SEED}"
)

tokenized_input = tokenizer([input_seq], return_tensors='pt', padding=True)
predicted_logits = model(**tokenized_input)

# Simple argmax - no thresholding.
argmax = predicted_logits[0].argmax(2)[0][1:-1].cpu().numpy()
indices = np.argwhere(argmax).flatten()

predicted_sequence = ''

for i, s in enumerate(input_seq):
    if i in indices:
        predicted_sequence += s
    else:
        predicted_sequence += '-'